# Preprocessing and storing DF in pickle
Perhaps we might Dwh Pat Dn Date later on. 

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os 
from datetime import timedelta
pd.set_option('display.max_columns', None)

data_path = str(Path(os.getcwd())) + "/data/"

def write_to_pickle(dataframe, name):
    dataframe.to_pickle(data_path + name + ".pickle")
def read_from_pickle(name): 
    return pd.read_pickle(data_path + name + ".pickle")

In [2]:
bio = pd.read_excel('data/20170713_Marc Gliomes_Bio.xlsx', index_col = None)
clin = pd.read_excel('data/20170713_Marc Gliomes_Clin.xlsx', index_col = None)

In [ ]:
b = bio.loc[bio['Pat Nip']==4406078178,'Dwh Prv Date Chir']
c = clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date']

#c.iloc[:len(c)]=1
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'] = c
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'] 
#print(clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'].iloc[i])
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date']    
#bio['Dwh Prv Date Chir'] = bio['Dwh Prv Date Chir'].apply(lambda x: x.to_pydatetime())
#bio.loc[2,'Dwh Prv Date Chir'].to_pydatetime()-timedelta(days=1)
#clin['Clin Chir Date'] = clin['Clin Chir Date'].apply(lambda x: x.to_pydatetime() + timedelta(days=1))
#bio['Dwh Prv Date Chir'] = bio['Dwh Prv Date Chir'].apply(lambda x: x.to_pydatetime())

Fix the surgery and clinical date so that they match

In [3]:
# Make the two surgery dates match 
# You are taking copies of the dataframe with loc and iloc, so you need to pay attention when you do assignment
# Can definitely remove one or both 'for' loops
for val in np.unique(clin['Pat Nip']): 
    bio_data = bio.loc[bio['Pat Nip']==val,'Dwh Prv Date Chir']
    clin_data = clin.loc[clin['Pat Nip']==val,'Clin Chir Date']
    lc = len(clin_data)
    lb = len(bio_data)
    if lc <=lb:
        for i in range(0,lc):
            clin_data.iloc[i] = bio_data.iloc[i]        
    else: 
        for i in range(0,lb):
            clin_data.iloc[i] = bio_data.iloc[i]
    clin.loc[clin['Pat Nip']==val,'Clin Chir Date'] = clin_data        

In [4]:
df = pd.merge(bio, clin, left_on = ['Pat Nip','Dwh Prv Date Chir'], right_on = ['Pat Nip','Clin Chir Date'],how='outer')
df.columns

Index(['Dwh Prv Nip', 'Dwh Pat Consentement', 'Pat Nip', 'Pat Sexe_x',
       'Pat Date Naissance_x', 'Dwh Pat Dn Date_x', 'Dwh Pat Date Deces_x',
       'Dwh Prv Date Chir', 'Dwh Prv Diag Histologie', 'Grade', 'Dwh Gen Idh1',
       'Dwh Gen Idh2', 'Dwh Gen Htert C228T', 'Dwh Gen Htert C250T',
       'Dwh Gen P53', 'Dwh Gen Mgmt', 'Dwh Gen Egfr', 'Dwh Gen Mdm2',
       'Dwh Gen Cdk4', 'Dwh Gen P16', 'Dwh Ihc Atrx', 'Dwh Chr Ch1P',
       'Dwh Chr Ch19Q', 'Dwh Chr Ch10P', 'Dwh Chr Ch10Q', 'Dwh Chr Ch9P',
       'Dwh Chr Ch9Q', 'Dwh Pat Tumeur Siege', 'Dwh Pat Tumeur Position',
       'Pat Sexe_y', 'Pat Date Naissance_y', 'Dwh Pat Dn Date_y',
       'Dwh Pat Date Deces_y', 'Clin Diag Histologie', 'Grade histo',
       'Clin Chir Date', 'Clin Evolution', 'Clin Type Chirurgie',
       'Clin Rx Date', 'Dwh Chm Date Debut'],
      dtype='object')

Before merging sexes, birthdays and deaths

In [5]:
# Can check before merging the whole thing as well. 
#bio['Pat Sexe'].isnull().value_counts()
print(df['Pat Sexe_x'].isnull().value_counts())
print(df['Pat Date Naissance_x'].isnull().value_counts())
print(df['Dwh Pat Date Deces_x'].isnull().value_counts())

False    7456
True      369
Name: Pat Sexe_x, dtype: int64
False    7725
True      100
Name: Pat Date Naissance_x, dtype: int64
True     4921
False    2904
Name: Dwh Pat Date Deces_x, dtype: int64


# Copy all _y columns into _x: sex, birthday and death, clinical histology, grade histology

In [5]:
for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Pat Sexe_x']
    y_data = df.loc[df['Pat Nip']==val,'Pat Sexe_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Pat Sexe_x'] = x_data

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Pat Date Naissance_x']
    y_data = df.loc[df['Pat Nip']==val,'Pat Date Naissance_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Pat Date Naissance_x'] = x_data    

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_x']
    y_data = df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_x'] = x_data   
    
for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Prv Diag Histologie']
    y_data = df.loc[df['Pat Nip']==val,'Clin Diag Histologie']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Prv Diag Histologie'] = x_data   

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Grade']
    y_data = df.loc[df['Pat Nip']==val,'Grade histo']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Grade'] = x_data   

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_x']
    y_data = df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_x'] = x_data   


In [6]:
# A small improvement
print(df['Pat Sexe_x'].isnull().value_counts())
print(df['Pat Date Naissance_x'].isnull().value_counts())
print(df['Dwh Pat Date Deces_x'].isnull().value_counts())
print(df['Dwh Prv Diag Histologie'].isnull().value_counts())
# Testing for empty values with an example to make sure merge is correct 
#clin.loc[clin['Pat Nip']==1111112776, 'Pat Sexe'].values
#clin['Pat Sexe'].value_counts()
#clin.loc[clin['Pat Sexe'].isnull()==True,'Pat Nip']

False    7493
True      332
Name: Pat Sexe_x, dtype: int64
False    7762
True       63
Name: Pat Date Naissance_x, dtype: int64
True     4908
False    2917
Name: Dwh Pat Date Deces_x, dtype: int64
False    7825
Name: Dwh Prv Diag Histologie, dtype: int64


In [7]:
df.rename(columns = {'Pat Nip':'ID','Pat Sexe_x':'Sex','Dwh Pat Dn Date_x':'Diagnostic_date','Pat Date Naissance_x':'DoB','Dwh Pat Date Deces_x':'Death_date',
                      'Dwh Prv Date Chir':'Surgery_date', 'Dwh Prv Diag Histologie':'Tumor_type','Grade':'Tumor_grade',
                      'Dwh Gen Idh1':'Gene_Idh1', 'Dwh Gen Idh2':'Gene_Idh2','Dwh Gen Htert C228T':'Gene_Htert_C228T', 
                      'Dwh Gen Htert C250T':'Gene_Htert_C250T', "Dwh Gen P53":'Gene_P53', 'Dwh Gen Mgmt':'Gene_Mgmt', 
                      'Dwh Gen Egfr':'Gene_Egfr', 'Dwh Gen Mdm2':'Gene_Mdm2','Dwh Gen Cdk4':'Gene_Cdk4',
                      'Dwh Gen P16':'Gene_P16', 'Dwh Ihc Atrx':'Gene_Ihc_Atrx', 'Dwh Chr Ch1P':'Gene_Ch1P',
                      'Dwh Chr Ch19Q':'Gene_Ch19Q', 'Dwh Chr Ch10P':'Gene_Ch10P', 'Dwh Chr Ch10Q':'Gene_Ch10Q', 'Dwh Chr Ch9P':'Gene_Ch9P',
                      'Dwh Chr Ch9Q':'Gene_Ch9Q', 'Dwh Pat Tumeur Siege':'Tumor_Location', 'Dwh Pat Tumeur Position':'Tumor_Position',
                      'Clin Evolution':'Clinical_evolution', 'Clin Type Chirurgie':'Surgery_type',
                      'Clin Rx Date':'Rx_date', 'Dwh Chm Date Debut':'Chemo_start_date'}, inplace = True)
df.drop(['Dwh Pat Dn Date_y','Pat Sexe_y','Dwh Pat Date Deces_y','Pat Date Naissance_y'], axis=1,inplace=True)
df.drop('Dwh Pat Consentement', axis=1, inplace=True)
df.drop('Dwh Prv Nip', axis=1, inplace=True)
df.drop(['Clin Diag Histologie', 'Grade histo','Clin Chir Date',],axis=1,inplace=True)

# Diagnostic very sparse, 2000 entries out of 7000
#interesting to analyze surgery date, all other things being equal. or any other proxy of impact of surgeon?
#what is the right measure of survival? date of diagnosis? date of surgery? none marks the actual 'beginning'. survival
#post diagnosis after controlling for tumor stage?

In [52]:
df.rename(columns = {'Sex':'Gender'},inplace = True) # forgot about this one
write_to_pickle(df,'post_merge_data')
df.head(2)
#df = read_from_pickle('post_merge_data')

,ID,Gender,DoB,Diagnostic_date,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date
0,4,M,1940-05-22,NaN,2001-02-22,2000-05-30,glioblastome IV,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT
1,5,M,1947-10-15,NaN,1999-12-15,1999-05-29,glioblastome IV,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sus-T hémisp,G,TT ini,biopsie,1999-07-12,1999-06-29


In [69]:
len(df.columns)
#df[df.ID ==4406078178]
df.Gene_Idh2.value_counts()

NORMAL    3941
NC        3031
ALTERE     106
Name: Gene_Idh2, dtype: int64

In [44]:
df.Death_date.isnull().value_counts()

True     4908
False    2917
Name: Death_date, dtype: int64

# End of merging. Next we deal with nan/empty values

In [ ]:
#Multiple entries for the same person, can we treat them independently?
#df.loc[df.ID==4204007149]

In [76]:
df_clean = df.copy(deep=True)
len(df_clean.columns)
df_clean = df_clean[pd.notnull(df.Gene_Idh2)]
df_clean = df_clean[pd.notnull(df.Gene_Idh1)]

/Users/patricfulop/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [97]:
df_final = df_clean

In [115]:
df_final.Gene_Ch9P.value_counts()

NON PERTE    2272
PARTIEL       768
PERDU         666
GAIN           91
Name: Gene_Ch9P, dtype: int64

In [57]:
for col in df_clean.columns:
     print('{0}\n  {1}\n'.format(col,df_clean[col].isnull().value_counts()))

ID
  False    7825
Name: ID, dtype: int64

Sex
  False    7493
True      332
Name: Sex, dtype: int64

DoB
  False    7762
True       63
Name: DoB, dtype: int64

Death_date
  True     4908
False    2917
Name: Death_date, dtype: int64

Surgery_date
  False    7356
True      469
Name: Surgery_date, dtype: int64

Tumor_type
  False    7825
Name: Tumor_type, dtype: int64

Tumor_grade
  False    7825
Name: Tumor_grade, dtype: int64

Gene_Idh1
  False    7327
True      498
Name: Gene_Idh1, dtype: int64

Gene_Idh2
  False    7078
True      747
Name: Gene_Idh2, dtype: int64

Gene_Htert_C228T
  False    4336
True     3489
Name: Gene_Htert_C228T, dtype: int64

Gene_Htert_C250T
  False    4333
True     3492
Name: Gene_Htert_C250T, dtype: int64

Gene_P53
  False    6495
True     1330
Name: Gene_P53, dtype: int64

Gene_Mgmt
  True     6508
False    1317
Name: Gene_Mgmt, dtype: int64

Gene_Egfr
  False    4487
True     3338
Name: Gene_Egfr, dtype: int64

Gene_Mdm2
  False    4524
True     3301
Name: 

In [ ]:
bio_asNumeric['Dwh Prv Nip'] = pd.to_numeric(bio_asNumeric['Dwh Prv Nip'], errors='coerce').astype('category')
bio_asNumeric['Dwh Pat Consentement'] = pd.to_numeric(bio_asNumeric['Dwh Pat Consentement'], errors = 'coerce').astype('category')
bio_asNumeric['Pat Nip'] = pd.to_numeric(bio_asNumeric['Pat Nip'], errors='coerce').astype('category')
bio_asNumeric['Pat Sexe'] = bio_asNumeric['Pat Sexe'].astype('category')
bio_asNumeric['Pat Date Naissance'] = pd.to_datetime(bio_asNumeric['Pat Date Naissance'])
bio_asNumeric['Dwh Pat Dn Date'] = pd.to_datetime(bio_asNumeric['Dwh Pat Date Deces'])
bio_asNumeric['Dwh Pat Date Deces'] = pd.to_datetime(bio_asNumeric['Dwh Pat Date Deces'])
bio_asNumeric['Dwh Prv Date Chir'] = pd.to_datetime(bio_asNumeric['Dwh Prv Date Chir'])
bio_asNumeric['Grade'] = pd.to_numeric(bio_asNumeric['Grade'], errors='coerce').astype('category')
bio_asNumeric['Dwh Pat Tumeur Position'] = bio_asNumeric['Dwh Pat Tumeur Position'].astype('category')

In [163]:
df = [lambda x: x for x in pat_date_naissance_y_null.index if x not in pat_date_naissance_x_null.index]

In [165]:
df_1 = [lambda x: x for x in pat_date_naissance_x_null.index if x not in pat_date_naissance_y_null.index]

In [171]:
df_2 = [lambda x: x for x in dwh_pat_dn_date_x_null.index if x not in dwh_pat_dn_date_y_null.index]

In [177]:
df_3 = [lambda x: x for x in dwh_pat_date_deces_x_null.index if x not in dwh_pat_date_deces_y_null.index]